##### In this notebook I will go over the code/syntax for the "Hotel Problem"! 

##### The goal is to find the best neighborhood/location to build a hotel in Manhattan, New York

Import the libraries:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  21.50 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  29.95 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.47 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  20.19 MB/s
vincent-0.4.4- 100% |###################

Load the Data for New York city:

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

Transfrom data into Pandas and fill the dataframe:

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Now we get neighborhoods only in Manhattan:

In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Georgraphical coordinates for Manhattan:

In [7]:
address = 'Manhattan, NY'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


Visualize Manhattan with its Neighborhoods:

In [8]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Foursquare credentials defined in the hidden cell bellow:

In [9]:
LIMIT=100
radius=3000 #this is about 2 miles radius

In [10]:
# The code was removed by Watson Studio for sharing.

To explore the Neighborhoods in Manhattan we create the following function:

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [13]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3307, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


How many venues were returned by each neighborhood:

In [14]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,100,100,100,100,100,100
Carnegie Hill,100,100,100,100,100,100
Central Harlem,42,42,42,42,42,42
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


How about the unique categories:

In [15]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 332 uniques categories.


Analyze each neighborhood:

In [16]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Sh

Group the neighborhoods:

In [17]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Sh

Top 10 venues for each neighborhood:

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Italian Restaurant,Department Store,Clothing Store,Market,Burger Joint,Memorial Site
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Spa,Wine Shop,Bookstore,Japanese Restaurant,French Restaurant
2,Central Harlem,African Restaurant,Cosmetics Shop,French Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Pizza Place,Bookstore,Market
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bakery,Nightclub,American Restaurant,Art Gallery,Theater,Hotel,Seafood Restaurant
4,Chinatown,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Dim Sum Restaurant,Hotpot Restaurant,Salon / Barbershop,Noodle House,Bakery
5,Civic Center,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Yoga Studio,Sandwich Place,Spa,Park,Sporting Goods Shop,Cocktail Bar
6,Clinton,Theater,Coffee Shop,Italian Restaurant,American Restaurant,Gym / Fitness Center,Wine Shop,Gym,Hotel,Spa,French Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Café,Beer Bar,Taco Place,Street Art,Dance Studio
8,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Cocktail Bar,Speakeasy,Ramen Restaurant,Chinese Restaurant,Pizza Place,Korean Restaurant
9,Financial District,Coffee Shop,Hotel,Wine Shop,Bar,Gym,Steakhouse,Food Truck,Italian Restaurant,Pizza Place,Falafel Restaurant


##### In this step I would like to eliminate the neighborhoods which already have hotels as their top venues:

In [19]:
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['1st Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['2nd Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['3rd Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['4th Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['5th Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['6th Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['7th Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['8th Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['9th Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[neighborhoods_venues_sorted['10th Most Common Venue'] != 'Hotel']
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Spa,Wine Shop,Bookstore,Japanese Restaurant,French Restaurant
2,Central Harlem,African Restaurant,Cosmetics Shop,French Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Pizza Place,Bookstore,Market
4,Chinatown,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Dim Sum Restaurant,Hotpot Restaurant,Salon / Barbershop,Noodle House,Bakery
5,Civic Center,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Yoga Studio,Sandwich Place,Spa,Park,Sporting Goods Shop,Cocktail Bar
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Café,Beer Bar,Taco Place,Street Art,Dance Studio
8,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Cocktail Bar,Speakeasy,Ramen Restaurant,Chinese Restaurant,Pizza Place,Korean Restaurant
10,Flatiron,Italian Restaurant,Yoga Studio,Gym / Fitness Center,American Restaurant,New American Restaurant,Salon / Barbershop,Gym,Sporting Goods Shop,Women's Store,Japanese Restaurant
11,Gramercy,Italian Restaurant,Cocktail Bar,Wine Shop,Bagel Shop,Restaurant,Coffee Shop,Thrift / Vintage Store,American Restaurant,Grocery Store,Pizza Place
12,Greenwich Village,Italian Restaurant,French Restaurant,Clothing Store,Sushi Restaurant,Seafood Restaurant,Indian Restaurant,Chinese Restaurant,Café,Coffee Shop,Cocktail Bar
13,Hamilton Heights,Mexican Restaurant,Coffee Shop,Café,Deli / Bodega,Pizza Place,Bakery,Cocktail Bar,Sandwich Place,School,Chinese Restaurant


Now that the Neighborhoods with no hotels are detected we shall Cluster neighborhoods:

In [20]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 0, 0, 0, 0, 0, 4, 0, 3], dtype=int32)

Top 10 Venues for each Neighborhood:

In [21]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Discount Store,Coffee Shop,Yoga Studio,Pizza Place,Supplement Shop,Steakhouse,Shopping Mall,Shoe Store,Seafood Restaurant,Sandwich Place
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Dim Sum Restaurant,Hotpot Restaurant,Salon / Barbershop,Noodle House,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Mobile Phone Shop,Sandwich Place,Pizza Place,Park,Mexican Restaurant,Shoe Store,Supermarket,Caribbean Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Café,Mexican Restaurant,Pizza Place,Lounge,Bakery,Deli / Bodega,Park,Restaurant,Chinese Restaurant,Frozen Yogurt Shop
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Mexican Restaurant,Coffee Shop,Café,Deli / Bodega,Pizza Place,Bakery,Cocktail Bar,Sandwich Place,School,Chinese Restaurant


Visualize:

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's examine our five clusters and explore

In [23]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Dim Sum Restaurant,Hotpot Restaurant,Salon / Barbershop,Noodle House,Bakery
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Sandwich Place,Pizza Place,Park,Mexican Restaurant,Shoe Store,Supermarket,Caribbean Restaurant
3,Inwood,Café,Mexican Restaurant,Pizza Place,Lounge,Bakery,Deli / Bodega,Park,Restaurant,Chinese Restaurant,Frozen Yogurt Shop
4,Hamilton Heights,Mexican Restaurant,Coffee Shop,Café,Deli / Bodega,Pizza Place,Bakery,Cocktail Bar,Sandwich Place,School,Chinese Restaurant
5,Manhattanville,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Japanese Curry Restaurant,Sushi Restaurant,Supermarket,Burger Joint,Bus Station,Spanish Restaurant
6,Central Harlem,African Restaurant,Cosmetics Shop,French Restaurant,American Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Pizza Place,Bookstore,Market
8,Upper East Side,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym / Fitness Center,Sporting Goods Shop,Gym,Burger Joint,Mexican Restaurant,Cosmetics Shop
11,Roosevelt Island,Park,Sandwich Place,Greek Restaurant,Coffee Shop,Dry Cleaner,Liquor Store,Bus Station,Baseball Field,Plaza,Kosher Restaurant
12,Upper West Side,Italian Restaurant,Bar,Coffee Shop,Wine Bar,Bakery,Burger Joint,Indian Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Gym / Fitness Center


In [24]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Little Italy,Bakery,Café,Yoga Studio,Cocktail Bar,Bubble Tea Shop,Seafood Restaurant,Salon / Barbershop,Ice Cream Shop,Sandwich Place,Boutique


In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Spa,Wine Shop,Bookstore,Japanese Restaurant,French Restaurant


In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Discount Store,Coffee Shop,Yoga Studio,Pizza Place,Supplement Shop,Steakhouse,Shopping Mall,Shoe Store,Seafood Restaurant,Sandwich Place
9,Yorkville,Coffee Shop,Italian Restaurant,Gym,Bar,Pizza Place,Deli / Bodega,Sushi Restaurant,Mexican Restaurant,Japanese Restaurant,Bagel Shop
25,Manhattan Valley,Coffee Shop,Pizza Place,Yoga Studio,Mexican Restaurant,Spa,Café,Thai Restaurant,Bar,Playground,Deli / Bodega
28,Battery Park City,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Café,Beer Bar,Taco Place,Street Art,Dance Studio
13,Lincoln Square,Gym / Fitness Center,Theater,Concert Hall,Plaza,Italian Restaurant,Café,French Restaurant,Opera House,Indie Movie Theater,Performing Arts Venue
15,Midtown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Cocktail Bar,Speakeasy,Ramen Restaurant,Chinese Restaurant,Pizza Place,Korean Restaurant
20,Lower East Side,Coffee Shop,Café,Art Gallery,Cocktail Bar,Ramen Restaurant,Sandwich Place,Latin American Restaurant,Chinese Restaurant,Shoe Store,Japanese Restaurant
21,Tribeca,Italian Restaurant,American Restaurant,Spa,Park,Café,Coffee Shop,Wine Bar,Boutique,Gym,Greek Restaurant
33,Midtown South,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,Stuyvesant Town,Bar,Park,Playground,Baseball Field,Pet Service,Gas Station,Harbor / Marina,Beer Garden,Cocktail Bar,Coffee Shop


One of my main criteria for a hotel location is that it needs to be near a park (a common venue of park should be in the neighborhood):

So I am going to eliminate all the neighborhoods who do not have common venue value of park/scenic Lookout:

In [28]:
cluster1=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
mask = np.column_stack([cluster1[col].str.contains(r"Park", na=False) for col in cluster1])
park1=cluster1.loc[mask.any(axis=1)]
park1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Sandwich Place,Pizza Place,Park,Mexican Restaurant,Shoe Store,Supermarket,Caribbean Restaurant
3,Inwood,Café,Mexican Restaurant,Pizza Place,Lounge,Bakery,Deli / Bodega,Park,Restaurant,Chinese Restaurant,Frozen Yogurt Shop
11,Roosevelt Island,Park,Sandwich Place,Greek Restaurant,Coffee Shop,Dry Cleaner,Liquor Store,Bus Station,Baseball Field,Plaza,Kosher Restaurant
24,West Village,Italian Restaurant,Cosmetics Shop,New American Restaurant,Gastropub,Wine Bar,Jazz Club,Park,American Restaurant,French Restaurant,Bakery
26,Morningside Heights,Coffee Shop,Park,Bookstore,American Restaurant,Deli / Bodega,Food Truck,Burger Joint,Sandwich Place,Tennis Court,Pizza Place
32,Civic Center,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Yoga Studio,Sandwich Place,Spa,Park,Sporting Goods Shop,Cocktail Bar


In [29]:
cluster2=manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]
mask2 = np.column_stack([cluster2[col].str.contains(r"Park", na=False) for col in cluster2])
park2=cluster2.loc[mask2.any(axis=1)]
park2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Tribeca,Italian Restaurant,American Restaurant,Spa,Park,Café,Coffee Shop,Wine Bar,Boutique,Gym,Greek Restaurant
37,Stuyvesant Town,Bar,Park,Playground,Baseball Field,Pet Service,Gas Station,Harbor / Marina,Beer Garden,Cocktail Bar,Coffee Shop


In [30]:
candidate_frames=[park1,park2]

In [31]:
candidates= pd.concat(candidate_frames)
candidates

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Sandwich Place,Pizza Place,Park,Mexican Restaurant,Shoe Store,Supermarket,Caribbean Restaurant
3,Inwood,Café,Mexican Restaurant,Pizza Place,Lounge,Bakery,Deli / Bodega,Park,Restaurant,Chinese Restaurant,Frozen Yogurt Shop
11,Roosevelt Island,Park,Sandwich Place,Greek Restaurant,Coffee Shop,Dry Cleaner,Liquor Store,Bus Station,Baseball Field,Plaza,Kosher Restaurant
24,West Village,Italian Restaurant,Cosmetics Shop,New American Restaurant,Gastropub,Wine Bar,Jazz Club,Park,American Restaurant,French Restaurant,Bakery
26,Morningside Heights,Coffee Shop,Park,Bookstore,American Restaurant,Deli / Bodega,Food Truck,Burger Joint,Sandwich Place,Tennis Court,Pizza Place
32,Civic Center,Gym / Fitness Center,Italian Restaurant,Bakery,French Restaurant,Yoga Studio,Sandwich Place,Spa,Park,Sporting Goods Shop,Cocktail Bar
21,Tribeca,Italian Restaurant,American Restaurant,Spa,Park,Café,Coffee Shop,Wine Bar,Boutique,Gym,Greek Restaurant
37,Stuyvesant Town,Bar,Park,Playground,Baseball Field,Pet Service,Gas Station,Harbor / Marina,Beer Garden,Cocktail Bar,Coffee Shop


These are our top 8 candidates who have common venue of "parks".

We now look at our candidate neighborhoods in terms of their top 10 frequent venues and their values:

In [32]:
num_top_venues = 10

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.08
1                Park  0.07
2               Hotel  0.05
3           Wine Shop  0.03
4  Italian Restaurant  0.03
5              Market  0.02
6      Sandwich Place  0.02
7      Ice Cream Shop  0.02
8      Clothing Store  0.02
9                 Gym  0.02


----Carnegie Hill----
                 venue  freq
0          Pizza Place  0.06
1       Cosmetics Shop  0.05
2          Coffee Shop  0.05
3                 Café  0.04
4  Japanese Restaurant  0.03
5                  Bar  0.03
6            Wine Shop  0.03
7                  Spa  0.03
8                  Gym  0.03
9    French Restaurant  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1    Seafood Restaurant  0.05
2   American Restaurant  0.05
3    Chinese Restaurant  0.05
4        Cosmetics Shop  0.05
5     French Restaurant  0.05
6  Gym / Fitness Center  0.05
7               Library  0.02
8             Bookstore  0.0

I would like to analyze and further explore in details 3 of the 8 candidates with higher density of restaurants/cafe and shops/attractions:

I would first explore the West Village neighborhood:

In [33]:
address = 'West Village, Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Village, Manhattan, NY are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of West Village, Manhattan, NY are 40.7352405, -74.0046133971969.


In terms of restaurants:

In [34]:
search_query = 'restaurant'
radius = 800
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ICT13051HFQ5WZ2GFXFVP2O1IQINIRXBPXG1Z3M3USEX1YA4&client_secret=RTMWLMFPCW5F5UZASC1C1ZBIQJJZ3VNFI2RTOJPMX3Z1TARO&ll=40.7352405,-74.0046133971969&v=20180605&query=restaurant&radius=800&limit=100'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c36240f1ed2194390b4590b'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d147941735',
      'name': 'Diner',
      'pluralName': 'Diners',
      'primary': True,
      'shortName': 'Diner'}],
    'delivery': {'id': '64940',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/waverly-restaurant-385-6th-ave-new-york/64940?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=64940'},
    'hasPerk': False,
    'id': '43bfd385f964a520232d1fe3',
    'location': {'address': '385 Avenue of the Americas',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'at W

In [36]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Waverly Restaurant,Diner,385 Avenue of the Americas,US,New York,United States,at Waverly Pl.,452,"[385 Avenue of the Americas (at Waverly Pl.), ...","[{'label': 'display', 'lat': 40.73301202815689...",40.733012,-74.000124,NaN,10014,NY,43bfd385f964a520232d1fe3
1,Sevilla Restaurant,Paella Restaurant,62 Charles St,US,New York,United States,at W 4th St,136,"[62 Charles St (at W 4th St), New York, NY 100...","[{'label': 'display', 'lat': 40.73486645783087...",40.734866,-74.003073,NaN,10014,NY,3fd66200f964a5201ae31ee3
2,RH Rooftop Restaurant,New American Restaurant,9 9th Ave,US,New York,United States,Little W 12th,534,"[9 9th Ave (Little W 12th), New York, NY 10014...","[{'label': 'display', 'lat': 40.73985, 'lng': ...",40.739850,-74.006390,NaN,10014,NY,5b906e3c1953f3002c880711
3,Spain Restaurant,Tapas Restaurant,113 W 13th St,US,New York,United States,at 6th Ave.,592,"[113 W 13th St (at 6th Ave.), New York, NY 100...","[{'label': 'display', 'lat': 40.73703527143349...",40.737035,-73.998004,NaN,10011,NY,3fd66200f964a520d0e51ee3
4,Salam Restaurant,Middle Eastern Restaurant,104 W 13th St,US,New York,United States,NaN,585,"[104 W 13th St, New York, NY 10011, United Sta...","[{'label': 'display', 'lat': 40.73682672786571...",40.736827,-73.998000,NaN,10011,NY,4b9ef1f0f964a5200e0c37e3
5,Revel Restaurant and Garden,American Restaurant,10 Little West 12th St,US,New York,United States,at 9th Ave. & Greenwich St.,532,[10 Little West 12th St (at 9th Ave. & Greenwi...,"[{'label': 'display', 'lat': 40.73969884138106...",40.739699,-74.006914,NaN,10014,NY,46dda121f964a520924a1fe3
6,Porto-Bello Restaurant,Italian Restaurant,208 Thompson St,US,New York,United States,at Bleecker St,845,"[208 Thompson St (at Bleecker St), New York, N...","[{'label': 'display', 'lat': 40.72875728893594...",40.728757,-73.999400,NaN,10012,NY,4afe06a9f964a520302d22e3
7,Panchito's Mexican Restaurant,Mexican Restaurant,105 Macdougal St,US,New York,United States,btwn Bleecker St. & Minetta Ln.,695,[105 Macdougal St (btwn Bleecker St. & Minetta...,"[{'label': 'display', 'lat': 40.72974217555018...",40.729742,-74.000698,NaN,10012,NY,4539fe02f964a520de3b1fe3
8,Rocky's Pizza Restaurant,Pizza Place,304 W 14th St,US,New York,United States,btwn 8th & 9th Ave.,518,"[304 W 14th St (btwn 8th & 9th Ave.), New York...","[{'label': 'display', 'lat': 40.739742, 'lng':...",40.739742,-74.003013,NaN,10014,NY,4c6f6888d97fa1434428f3ca
9,St Tropez Restaurant & Wine Bar,Wine Bar,304 W 4th St,US,New York,United States,Bank,213,"[304 W 4th St (Bank), New York, NY 10014, Unit...","[{'label': 'display', 'lat': 40.73705858350001...",40.737059,-74.003795,West Village,10014,NY,5a294d59dd12f82944c92e1d


In [37]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='West Village, Manhattan, NY',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Now the "Tribeca" area:

In [38]:
address = 'Tribeca, Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tribeca, Manhattan, NY are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Tribeca, Manhattan, NY are 40.7153802, -74.0093063.


In [39]:
search_query = 'restaurant'
radius = 800
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ICT13051HFQ5WZ2GFXFVP2O1IQINIRXBPXG1Z3M3USEX1YA4&client_secret=RTMWLMFPCW5F5UZASC1C1ZBIQJJZ3VNFI2RTOJPMX3Z1TARO&ll=40.7153802,-74.0093063&v=20180605&query=restaurant&radius=800&limit=100'

In [40]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3625e74c1f6740857bc770'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/wings_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d14c941735',
      'name': 'Wings Joint',
      'pluralName': 'Wings Joints',
      'primary': True,
      'shortName': 'Wings'}],
    'delivery': {'id': '299726',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/mudville-restaurant-126-chambers-st-new-york/299726?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=299726'},
    'hasPerk': False,
    'id': '45e5c256f964a52046431fe3',
    'location': {'address': '126 Chambers St',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStree

In [41]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Mudville Restaurant & Tap House,Wings Joint,126 Chambers St,US,New York,United States,btwn W Broadway & Church St,36,[126 Chambers St (btwn W Broadway & Church St)...,"[{'label': 'display', 'lat': 40.71533575723845...",40.715336,-74.008881,NaN,10007,NY,45e5c256f964a52046431fe3
1,O'Hara's Restaurant & Pub,Pub,120 Cedar St,US,New York,United States,at Greenwich St.,679,"[120 Cedar St (at Greenwich St.), New York, NY...","[{'label': 'display', 'lat': 40.70989378141622...",40.709894,-74.012836,NaN,10006,NY,49f125dcf964a52091691fe3
2,Amore's Pizza Restaurant,Pizza Place,147 Chambers St,US,New York,United States,Hudson Street,72,"[147 Chambers St (Hudson Street), New York, NY...","[{'label': 'display', 'lat': 40.71585960614924...",40.715860,-74.009888,NaN,10007,NY,4bc238adf8219c744286b410
3,Bo Ky Restaurant 波記潮州小食,Chinese Restaurant,80 Bayard St,US,New York,United States,at Mott St,898,"[80 Bayard St (at Mott St), New York, NY 10013...","[{'label': 'display', 'lat': 40.71569636637641...",40.715696,-73.998667,NaN,10013,NY,4a00df67f964a520ba701fe3
4,Hop Shing Restaurant 合誠茶樓,Dim Sum Restaurant,9 Chatham Sq,US,New York,United States,at E Broadway,973,"[9 Chatham Sq (at E Broadway), New York, NY 10...","[{'label': 'display', 'lat': 40.713932, 'lng':...",40.713932,-73.997930,NaN,10038,NY,4b38c437f964a520245025e3
5,New Shezan Restaurant,Middle Eastern Restaurant,183 Church St,US,New York,United States,btwn Duane & Reade St.,181,"[183 Church St (btwn Duane & Reade St.), New Y...","[{'label': 'display', 'lat': 40.71578949233164...",40.715789,-74.007227,NaN,10007,NY,4c4890da1879c9b6cce6e143
6,Sun Sai Gai Restaurant,Chinese Restaurant,220 Canal St,US,New York,United States,at Baxter St,863,"[220 Canal St (at Baxter St), New York, NY 100...","[{'label': 'display', 'lat': 40.71736941942955...",40.717369,-73.999415,NaN,10013,NY,4a81ac53f964a5203af71fe3
7,Chatham Seafood Restaurant,Dim Sum Restaurant,6 Chatham Sq,US,New York,United States,btwn Mott & Doyers St,946,"[6 Chatham Sq (btwn Mott & Doyers St), New Yor...","[{'label': 'display', 'lat': 40.71388466954485...",40.713885,-73.998266,NaN,10038,NY,49c68407f964a52052571fe3
8,TJ Byrnes Bar and Restaurant,Restaurant,77 Fulton St,US,New York,United States,Gold St,829,"[77 Fulton St (Gold St), New York, NY 10038, U...","[{'label': 'display', 'lat': 40.70923312629616...",40.709233,-74.003747,NaN,10038,NY,4b4fdfc8f964a520801827e3
9,Deluxe Green Bo Restaurant,Chinese Restaurant,66 Bayard St,US,New York,United States,btwn Elizabeth & Mott St.,942,"[66 Bayard St (btwn Elizabeth & Mott St.), New...","[{'label': 'display', 'lat': 40.71554491813315...",40.715545,-73.998137,NaN,10013,NY,3fd66200f964a520ceea1ee3


In [42]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Tribeca, Manhattan, NY',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Now "Inwood" area:

In [43]:
address = 'Inwood, Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Inwood, Manhattan, NY are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Inwood, Manhattan, NY are 40.8692579, -73.9204949.


In [44]:
search_query = 'restaurant'
radius = 800
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ICT13051HFQ5WZ2GFXFVP2O1IQINIRXBPXG1Z3M3USEX1YA4&client_secret=RTMWLMFPCW5F5UZASC1C1ZBIQJJZ3VNFI2RTOJPMX3Z1TARO&ll=40.8692579,-73.9204949&v=20180605&query=restaurant&radius=800&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3627f6dd57970a33aa05cf'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/latinamerican_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1be941735',
      'name': 'Latin American Restaurant',
      'pluralName': 'Latin American Restaurants',
      'primary': True,
      'shortName': 'Latin American'}],
    'hasPerk': False,
    'id': '4cb9097c035d236aba86d14e',
    'location': {'address': '3855 10th Ave',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'distance': 534,
     'formattedAddress': ['3855 10th Ave',
      'New York, NY 10034',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.86457014083862,
       'lng': -73.919136}],
     'lat': 40.86457014083862,
     'lng': -73.919136,
     'postalCode': '10034',
     'state': 'NY'},
    'name': 'El Lina Restaurant',
    'referralId': 'v-1547053046'},
   {'categories

In [46]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,El Lina Restaurant,Latin American Restaurant,3855 10th Ave,US,New York,United States,NaN,534,"[3855 10th Ave, New York, NY 10034, United Sta...","[{'label': 'display', 'lat': 40.86457014083862...",40.864570,-73.919136,10034,NY,4cb9097c035d236aba86d14e
1,Amy's Restaurant,Dumpling Restaurant,586 W 207th St,US,New York,United States,Btnw Broadway and Vermilyea Ave,262,[586 W 207th St (Btnw Broadway and Vermilyea A...,"[{'label': 'display', 'lat': 40.86691721459921...",40.866917,-73.920844,10034,NY,4ba2b1faf964a5200d1138e3
2,Caridad Restaurant,Caribbean Restaurant,4799 Broadway,US,New York,United States,NaN,511,"[4799 Broadway, New York, NY 10034, United Sta...","[{'label': 'display', 'lat': 40.86649138124692...",40.866491,-73.925347,10034,NY,4be65857bcef2d7f512005e5
3,Capitol Restaurant,Diner,207th and Broadway,US,New York,United States,Broadway,193,"[207th and Broadway (Broadway), New York, NY 1...","[{'label': 'display', 'lat': 40.86768070261630...",40.867681,-73.921459,10034,NY,4bf716e013aed13a67cdeaf7
4,Ceci Restaurant,Latin American Restaurant,78 Sherman Ave,US,New York,United States,btwn Thayer St & Dyckman St,816,"[78 Sherman Ave (btwn Thayer St & Dyckman St),...","[{'label': 'display', 'lat': 40.86353731603869...",40.863537,-73.926565,10040,NY,4b47ce50f964a5207f3f26e3
5,Seawalk Restaurant,Seafood Restaurant,261 Dyckman St,US,New York,United States,NaN,720,"[261 Dyckman St, New York, NY 10034, United St...","[{'label': 'display', 'lat': 40.86665009012302...",40.866650,-73.928328,10034,NY,5910c97928374e21303f6580
6,Mina Bar & Restaurant,Hookah Bar,3859 10th Ave,US,New York,United States,206th St,608,"[3859 10th Ave (206th St), New York, NY 10034,...","[{'label': 'display', 'lat': 40.863907, 'lng':...",40.863907,-73.919020,10034,NY,5b52709c835c9a00395e8dbc
7,El Tina Seafood Restaurant,Seafood Restaurant,500 W 207th St,US,New York,United States,NaN,546,"[500 W 207th St, New York, NY 10034, United St...","[{'label': 'display', 'lat': 40.86445981289547...",40.864460,-73.919098,10034,NY,4bb93e047421a5933fa7c240
8,Lebron Restaurant Equipment & Supplies,Miscellaneous Shop,3835 9th Ave,US,New York,United States,205th St,762,"[3835 9th Ave (205th St), New York, NY 10034, ...","[{'label': 'display', 'lat': 40.86273013988233...",40.862730,-73.917746,10034,NY,4c2cd52b57a9c9b6ab0cf867
9,Lin Xian Biao Restaurant,Food,595 W 207th St,US,New York,United States,NaN,228,"[595 W 207th St, New York, NY 10034, United St...","[{'label': 'display', 'lat': 40.867226, 'lng':...",40.867226,-73.920913,10034,NY,4f32102d19833175d60cd860


In [48]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Inwood, Manhattan, NY',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

I also would like to compare the concentration of shops in each neighborhood:

Starting with "West Village":

In [49]:
address = 'West Village, Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Village, Manhattan, NY are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Inwood, Manhattan, NY are 40.7352405, -74.0046133971969.


In [50]:
search_query = 'shop'
radius = 800
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ICT13051HFQ5WZ2GFXFVP2O1IQINIRXBPXG1Z3M3USEX1YA4&client_secret=RTMWLMFPCW5F5UZASC1C1ZBIQJJZ3VNFI2RTOJPMX3Z1TARO&ll=40.7352405,-74.0046133971969&v=20180605&query=shop&radius=800&limit=100'

In [51]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c36292ef594df3abfc441ca'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/dessert_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1d0941735',
      'name': 'Dessert Shop',
      'pluralName': 'Dessert Shops',
      'primary': True,
      'shortName': 'Desserts'}],
    'hasPerk': False,
    'id': '4a80e191f964a52065f61fe3',
    'location': {'address': '243 Bleecker St',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'at Leroy St',
     'distance': 517,
     'formattedAddress': ['243 Bleecker St (at Leroy St)',
      'New York, NY 10014',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.7308059022432,
       'lng': -74.00276327968298}],
     'lat': 40.7308059022432,
     'lng': -74.00276327968298,
     'postalCode': '10014',
     'state': 'NY'},
    'name': 'Pasticceria Rocco - Pastry Shop and Espres

In [52]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pasticceria Rocco - Pastry Shop and Espresso Cafe,Dessert Shop,243 Bleecker St,US,New York,United States,at Leroy St,517,"[243 Bleecker St (at Leroy St), New York, NY 1...","[{'label': 'display', 'lat': 40.7308059022432,...",40.730806,-74.002763,NaN,10014,NY,4a80e191f964a52065f61fe3
1,Goorin Bros. Hat Shop - West Village,Accessories Store,337 Bleecker St,US,New York,United States,btwn W 10th & Christopher St.,182,[337 Bleecker St (btwn W 10th & Christopher St...,"[{'label': 'display', 'lat': 40.73360879617734...",40.733609,-74.004411,NaN,10014,NY,4e8d02c76da1d5bdcf09bbcd
2,Housing Works Thrift Shop,Thrift / Vintage Store,245 W 10th St,US,New York,United States,at Hudson St,187,"[245 W 10th St (at Hudson St), New York, NY 10...","[{'label': 'display', 'lat': 40.73385479510609...",40.733855,-74.005885,NaN,10014,NY,4aa81aecf964a5205a4f20e3
3,Sammy's Noodle Shop,Chinese Restaurant,453-461 Avenue of the Americas,US,New York,United States,btwn 10th St & 11th St,516,[453-461 Avenue of the Americas (btwn 10th St ...,"[{'label': 'display', 'lat': 40.73514392366530...",40.735144,-73.998492,NaN,10011,NY,3fd66200f964a520f2e91ee3
4,Housing Works Thrift Shop,Thrift / Vintage Store,143 W 17th St,US,New York,United States,btwn 6th & 7th Ave,822,"[143 W 17th St (btwn 6th & 7th Ave), New York,...","[{'label': 'display', 'lat': 40.74009088604733...",40.740091,-73.997254,Chelsea,10011,NY,4ab10f76f964a520e76720e3
5,Num Pang Sandwich Shop,Sandwich Place,75 9th Ave,US,New York,United States,at W 15th St,791,"[75 9th Ave (at W 15th St), New York, NY 10011...","[{'label': 'display', 'lat': 40.74224903617023...",40.742249,-74.006172,NaN,10011,NY,51856167498e6737663ecd44
6,Kikkerland Shop NYC,Gift Shop,493 Avenue of the Americas,US,New York,United States,12th and 13th,593,"[493 Avenue of the Americas (12th and 13th), N...","[{'label': 'display', 'lat': 40.736357, 'lng':...",40.736357,-73.997736,NaN,10011,NY,528f67bf11d24cd6cf3685de
7,Neighborhood Cut and Shave Barber Shop,Salon / Barbershop,37A Bedford St,US,New York,United States,Downing and Carmine (WEST VILLAGE),618,[37A Bedford St (Downing and Carmine (WEST VIL...,"[{'label': 'display', 'lat': 40.72971736816688...",40.729717,-74.003853,NaN,10014,NY,4dbc5844b3dcc563f2d5b053
8,The Taco Shop,Taco Place,166 W 4th St,US,New York,United States,btwn Cornelia & Jones St,454,"[166 W 4th St (btwn Cornelia & Jones St), New ...","[{'label': 'display', 'lat': 40.73196898142531...",40.731969,-74.001394,NaN,10014,NY,4fa32864e4b09d4d43e7fd7e
9,Big Gay Ice Cream Shop,Ice Cream Shop,61 Grove St,US,New York,United States,at 7th Ave S,265,"[61 Grove St (at 7th Ave S), New York, NY 1001...","[{'label': 'display', 'lat': 40.73319335248443...",40.733193,-74.002988,NaN,10014,NY,50ba7adbe4b036209ec2a3bd


In [54]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='West Village, Manhattan, NY',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Now the "Tribeca" area:

In [55]:
address = 'Tribeca, Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tribeca, Manhattan, NY are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Tribeca, Manhattan, NY are 40.7153802, -74.0093063.


In [56]:
search_query = 'shop'
radius = 800
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ICT13051HFQ5WZ2GFXFVP2O1IQINIRXBPXG1Z3M3USEX1YA4&client_secret=RTMWLMFPCW5F5UZASC1C1ZBIQJJZ3VNFI2RTOJPMX3Z1TARO&ll=40.7153802,-74.0093063&v=20180605&query=shop&radius=800&limit=100'

In [57]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c362a016a60713c2d416c67'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/salon_barber_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d110951735',
      'name': 'Salon / Barbershop',
      'pluralName': 'Salons / Barbershops',
      'primary': True,
      'shortName': 'Salon / Barbershop'}],
    'hasPerk': False,
    'id': '52a0a3d411d2a9c50dff1d6e',
    'location': {'address': '46 Park Pl',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'Church Street',
     'distance': 220,
     'formattedAddress': ['46 Park Pl (Church Street)',
      'New York, NY 10007',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.7134161580909,
       'lng': -74.00961020690364}],
     'lat': 40.7134161580909,
     'lng': -74.00961020690364,
     'postalCode': '10007',
     'state': 'NY'},
    'name': "Ray's Barber Shop",

In [58]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Ray's Barber Shop,Salon / Barbershop,46 Park Pl,US,New York,United States,Church Street,220,"[46 Park Pl (Church Street), New York, NY 1000...","[{'label': 'display', 'lat': 40.7134161580909,...",40.713416,-74.009610,NaN,10007,NY,52a0a3d411d2a9c50dff1d6e
1,Housing Works Thrift Shop,Thrift / Vintage Store,119 Chambers St,US,New York,United States,btwn Broadway & Church St,59,"[119 Chambers St (btwn Broadway & Church St), ...","[{'label': 'display', 'lat': 40.71525006902321...",40.715250,-74.008625,NaN,10007,NY,4bc3a005f8219c74815eb610
2,West Side Coffee Shop,Latin American Restaurant,323 Church St,US,New York,United States,at Canal St.,701,"[323 Church St (at Canal St.), New York, NY 10...","[{'label': 'display', 'lat': 40.72017383765389...",40.720174,-74.003915,NaN,10013,NY,4a667506f964a52093c81fe3
3,Hong Kong Barber Shop,Salon / Barbershop,19 Doyers St,US,New York,United States,Bowery,952,"[19 Doyers St (Bowery), New York, NY 10013, Un...","[{'label': 'display', 'lat': 40.71471731921922...",40.714717,-73.998055,NaN,10013,NY,4b6de20cf964a52082982ce3
4,Smoke Shop,Smoke Shop,20 Vesey St,US,New York,United States,btwn Church St & Broadway,396,"[20 Vesey St (btwn Church St & Broadway), New ...","[{'label': 'display', 'lat': 40.71190417219223...",40.711904,-74.010305,NaN,10007,NY,502ab09be4b0cb6d8a90d016
5,Tribeca Cobbler Shop,Shoe Repair,390 Greenwich St,US,NaN,United States,NaN,610,"[390 Greenwich St, NY 10013, United States]","[{'label': 'display', 'lat': 40.720833, 'lng':...",40.720833,-74.010085,NaN,10013,New York,4e2476a2aeb70263954cb680
6,9/11 Museum Gift Shop,Gift Shop,180 Greenwich St,US,New York,United States,at Liberty St,516,"[180 Greenwich St (at Liberty St), New York, N...","[{'label': 'display', 'lat': 40.71164624294874...",40.711646,-74.012940,NaN,10007,NY,5376a2e011d20d02a0831221
7,NY Gift Shop,Gift Shop,234 Canal St,US,New York,United States,at Centre St.,407,"[234 Canal St (at Centre St.), New York, NY 10...","[{'label': 'display', 'lat': 40.71273261495395...",40.712733,-74.005978,NaN,10013,NY,4b79a5e8f964a52037082fe3
8,Potbelly Sandwich Shop,Sandwich Place,280 Broadway,US,New York,United States,btwn Reade & Chambers St,309,"[280 Broadway (btwn Reade & Chambers St), New ...","[{'label': 'display', 'lat': 40.71448, 'lng': ...",40.714480,-74.005842,NaN,10007,NY,4f4d2593e4b00b42c3f1060f
9,Num Pang Sandwich Shop,Sandwich Place,225 Liberty St,US,New York,United States,West Street,639,"[225 Liberty St (West Street), New York, NY 10...","[{'label': 'display', 'lat': 40.71272662965214...",40.712727,-74.016032,NaN,10281,NY,536937d4498e999c655b37c7


In [59]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Tribeca, Manhattan, NY',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

And "Inwood" area:

In [61]:
address = 'Inwood, Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Inwood, Manhattan, NY are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Inwood, Manhattan, NY are 40.8692579, -73.9204949.


In [62]:
search_query = 'shop'
radius = 800
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=ICT13051HFQ5WZ2GFXFVP2O1IQINIRXBPXG1Z3M3USEX1YA4&client_secret=RTMWLMFPCW5F5UZASC1C1ZBIQJJZ3VNFI2RTOJPMX3Z1TARO&ll=40.8692579,-73.9204949&v=20180605&query=shop&radius=800&limit=100'

In [63]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c362a794c1f67402e117d2f'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/diner_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d147941735',
      'name': 'Diner',
      'pluralName': 'Diners',
      'primary': True,
      'shortName': 'Diner'}],
    'hasPerk': False,
    'id': '4be88652947820a1e802b5db',
    'location': {'address': '634 W 207th St',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'btwn Broadway & Cooper St',
     'distance': 140,
     'formattedAddress': ['634 W 207th St (btwn Broadway & Cooper St)',
      'New York, NY 10034',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.86841792395646,
       'lng': -73.92174351734803}],
     'lat': 40.86841792395646,
     'lng': -73.92174351734803,
     'postalCode': '10034',
     'state': 'NY'},
    'name': "G's Coffee Shop",
    'referralId

In [64]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,G's Coffee Shop,Diner,634 W 207th St,US,New York,United States,btwn Broadway & Cooper St,140,"[634 W 207th St (btwn Broadway & Cooper St), N...","[{'label': 'display', 'lat': 40.86841792395646...",40.868418,-73.921744,NaN,10034,NY,4be88652947820a1e802b5db
1,MTA NYCT 207th Street Maintenance and Overhaul...,Government Building,3961 10th Ave. - 207 ST. Yard,US,New York,United States,NaN,362,"[3961 10th Ave. - 207 ST. Yard, New York, NY, ...","[{'label': 'display', 'lat': 40.86711177619066...",40.867112,-73.917258,NaN,NaN,NY,4d3835390f81b60c53bd3d3a
2,MTA - 207th Street Yard/Maintenance Facility/O...,Government Building,3961 10th Ave,US,New York,United States,at Isham St,517,"[3961 10th Ave (at Isham St), New York, NY 100...","[{'label': 'display', 'lat': 40.86468577887141...",40.864686,-73.919406,NaN,10034,NY,4bfbaca6f214a5936f6eaf0e
3,Shorty Style Unisex & Barber Shop,Salon / Barbershop,3852 10th Ave,US,New York,United States,206th St,702,"[3852 10th Ave (206th St), New York, NY 10034,...","[{'label': 'display', 'lat': 40.86299789943816...",40.862998,-73.919452,NaN,10034,NY,50f1b066e4b0c0d3a908f85a
4,Inwood Barber Shop,Salon / Barbershop,4915 Broadway,US,New York,United States,NaN,371,"[4915 Broadway, New York, NY 10034, United Sta...","[{'label': 'display', 'lat': 40.86754072832629...",40.867541,-73.924285,Inwood,10034,NY,4d14c7ec6d103704426a24bd
5,Los Taxistas Barber Shop,Salon / Barbershop,499 W 207 st,US,New York,United States,NaN,533,"[499 W 207 st, New York, NY, United States]","[{'label': 'display', 'lat': 40.864594, 'lng':...",40.864594,-73.919048,NaN,NaN,NY,4bd2132941b9ef3b9545fde5
6,Erik's Barber Shop,Cosmetics Shop,NaN,US,New York,United States,NaN,193,"[New York, NY 10034, United States]","[{'label': 'display', 'lat': 40.86754616864211...",40.867546,-73.920919,NaN,10034,NY,4bd9cfefd2cbc9289cfed1ad
7,Tread Bike Shop,Bike Shop,250 Dyckman St,US,New York,United States,btw Seamen & Payson,734,"[250 Dyckman St (btw Seamen & Payson), New Yor...","[{'label': 'display', 'lat': 40.86636345173913...",40.866363,-73.928338,NaN,10034,NY,4ba4eb06f964a520e6c238e3
8,Eric Barber Shop,Salon / Barbershop,609 West Broadway,US,New York,United States,The Hights,370,"[609 West Broadway (The Hights), New York, NY,...","[{'label': 'display', 'lat': 40.867041, 'lng':...",40.867041,-73.917209,NaN,NaN,NY,4eaad34d4901584489aaf0c1
9,Jct Coffee Shop,Food,5009 Broadway,US,New York,United States,NaN,306,"[5009 Broadway, New York, NY 10034, United Sta...","[{'label': 'display', 'lat': 40.869329, 'lng':...",40.869329,-73.916852,NaN,10034,NY,4f32453919836c91c7c7181e


In [66]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Inwood, Manhattan, NY',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

My top three candidates for opening up a hotel in a Manhattan Neighborhoods are: 1)Tribeca 2)Inwoods 3)West Village areas! 

All three of them are close/walking deistance to parks and landmarks as well as concentrated shopping areas, cafe, and restaurants. Also in those neighborhoods there are only handfull of hotels present (for example only 1 hotel in Inwoods area), so there is opportunity to open up a new one!